# Aide à la Recherche Clinique automatisée : extraction automatique des données à partir de compte-rendu

In [2]:
import pandas as pd
import pdfplumber
import os
import re 
import unicodedata

### PARTIE 1 : RÉCUPÉRATION

#### Récupération des comptes rendus PDF et lancement de l'extraction automatique de données 

Ici, je vais utiliser l'outil [pdfplumber](https://pypi.org/project/pdfplumber/) pour lire les fichiers PDF.
De cette manière, Je vais prendre chaque fichier du dossier et le lire par page.

Ensuite, je vais copier le contenu de chaque page dans un fichier texte.
Pour séparation des compte rendus, je vais utiliser la phrase "SERVICE D’ANATOMIE PATHOLOGIQUE ET DE NEUROPATHOLOGIE" comme séparateur, car il y a un espace entre la phrase "SERVICE D’ANATOMIE PATHOLOGIQUE ET DE NEUROPATHOLOGIE" et le texte du compte rendu. 

In [53]:
# Folder containing the PDF files to analyze
pdf_folder = "PDF - TRAINING"

# List of PDF files to scan
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

# Report number
report_number = 1

# Initialize the variable to store the content
all_reports_content = ""

for file in pdf_files:
    with pdfplumber.open(os.path.join(pdf_folder, file)) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            # Separation of reports based on "SERVICE D’ANATOMIE PATHOLOGIQUE ET DE NEUROPATHOLOGIE"
            reports = text.split("SERVICE D’ANATOMIE PATHOLOGIQUE ET DE NEUROPATHOLOGIE")
            
            for report in reports:
                if report.strip():  # If the text is not empty
                    report_content = f"\n------ COMPTE RENDU N°{report_number} ------\n"
                    report_content += report + "\n"
                    all_reports_content += report_content
                    report_number += 1

# Afficher le contenu complet
print(all_reports_content)


------ COMPTE RENDU N°1 ------
SERVICE D’ANATOMIE PATHOLOGIQUE ET DE
NEUROPATHOLOGIE - CHU - TIMONE
TUMOROTHEQUE ET BANQUE DE MUSCLES DE l’AP-HM
Professeur Laurent DANIEL
Pôle 18 : Biologie et Pathologie - N°FINESS : 130783293
264 rue Saint Pierre – 13385 MARSEILLE Cedex 05
Tél. 04 13 42 90 10 – Fax. 04 13 42 90 42 / 04 91 38 44 11
N° NIP :
UF : 0467 RICHARD MARIE-ALETH
-
Pr. RICHARD MARIE-ALETH
*HHH2325367*
CONSULT.PETITE CHIRURGIE DERMATO-TA
APHM TIMONE ADULTES
13385 MARSEILLE
_____________________________________________________________________________________
Prescripteur : Dr MOLINIER Préleveur : Dr MOLINIER
A. EXERESE CUTANEE : JAMBE GAUCHE
Renseignements : Face antéro-interne, lésion pigmentée hétérogène suspecte. Mélanome ? Exérèse sans marge
Exérèse cutanée – Formol – Orientation : non - Dimensions : 40 x 35 x 10 mm – Lésion : ~25 mm
Inclusion : 7 bloc(s) – représentatif(s) : A1 a A7
Description : involution lésionnelle, avec pigment dermique, malgré de multiples plans de cou

Ici on va "fusionner" les pages d'un même compte rendu en exploitant la regex "Page n/m" pour savoir si le compte rendu est composé d'une seule page ou de plusieurs.

In [54]:
# Folder containing the PDF files to analyze
pdf_folder = "PDF - TRAINING"

# List of PDF files to scan
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

# Report number
report_number = 1

# Initialize the variable to store the content
all_reports_content = ""

for file in pdf_files:
    file_text = ""  # Accumulate the text of all pages of files 
    with pdfplumber.open(os.path.join(pdf_folder, file)) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            file_text += text + "\n" # Add a line break between the text on each page

    # Checking the paging format to determine if the file should be treated as a single report or not
    pagination_match = re.search(r"Page 1/(\d+)", file_text)
    if pagination_match and pagination_match.group(1) != "1":
        # If "Page 1/n" where n > 1, treat all the text in the file as a single report
        report_content = f"\n------ COMPTE RENDU N°{report_number} ------\n"
        report_content += file_text
        all_reports_content += report_content + "\n"
        report_number += 1
    else:
        # Otherwise, treat each “SERVICE D’ANATOMIE PATHOLOGIQUE ET DE NEUROPATHOLOGIEE” as a separate report
        reports = file_text.split("SERVICE D’ANATOMIE PATHOLOGIQUE ET DE NEUROPATHOLOGIE")
        for report in reports:
            if report.strip():  # Vérifie si le texte n'est pas vide
                report_content = f"\n------ COMPTE RENDU N°{report_number} ------\n"
                report_content += report + "\n"
                all_reports_content += report_content
                report_number += 1

print(all_reports_content)


------ COMPTE RENDU N°1 ------
SERVICE D’ANATOMIE PATHOLOGIQUE ET DE
NEUROPATHOLOGIE - CHU - TIMONE
TUMOROTHEQUE ET BANQUE DE MUSCLES DE l’AP-HM
Professeur Laurent DANIEL
Pôle 18 : Biologie et Pathologie - N°FINESS : 130783293
264 rue Saint Pierre – 13385 MARSEILLE Cedex 05
Tél. 04 13 42 90 10 – Fax. 04 13 42 90 42 / 04 91 38 44 11
N° NIP :
UF : 0467 RICHARD MARIE-ALETH
-
Pr. RICHARD MARIE-ALETH
*HHH2325367*
CONSULT.PETITE CHIRURGIE DERMATO-TA
APHM TIMONE ADULTES
13385 MARSEILLE
_____________________________________________________________________________________
Prescripteur : Dr MOLINIER Préleveur : Dr MOLINIER
A. EXERESE CUTANEE : JAMBE GAUCHE
Renseignements : Face antéro-interne, lésion pigmentée hétérogène suspecte. Mélanome ? Exérèse sans marge
Exérèse cutanée – Formol – Orientation : non - Dimensions : 40 x 35 x 10 mm – Lésion : ~25 mm
Inclusion : 7 bloc(s) – représentatif(s) : A1 a A7
Description : involution lésionnelle, avec pigment dermique, malgré de multiples plans de cou

### PARTIE 2 : PRÉ-TRAITEMENT 

##### Premièrement on va prétraiter la ligne contenant "Prescripteur" et "Préleveur" pour uniformiser les cas où l'on a pas de professionnels dans les rapports pour ces deux champs ou alors quand on a des champs faux (C'est à dire qui ne sont pas sous la forme DR/Dr/dr/PR/Pr/pr NOM ou alors NOM).

D'abord on va afficher que les lignes contenant "Prescripteur" et "Préleveur" puis on va spliter les champs pour stocker la partie "NOM" du champs "Prescripteur : NOM" dans la variable "prescripteur_part", de même pour le champs "Préleveur : NOM" dans la variable "preleveur_part".

Lorsqu'on ne trouve pas de correspondance pour l'un ou l'autre alors on écrira "None" dans le champ correspondant.

In [55]:
 def extract_information(text):
    # Regular expressions to identify lines with "Prescripteur" or "Préleveur"
    prescripteur_pattern = re.compile(r'Prescripteur\s*:\s*(.*)')
    preleveur_pattern = re.compile(r'Préleveur\s*:\s*(.*)')
    # Errors pattern 
    first_errors = re.compile(r'[DR|dr|Dr|PR|Pr|pr] /')
    
    # Browse lines of text
    for line_number, line in enumerate(text.split('\n')):
        # Check if there is a match with "Prescripteur"
        prescripteur_match = prescripteur_pattern.match(line)
        if prescripteur_match:
            # Divide the line into two parts: before and after the "Prescripteur"
            prescripteur_part = prescripteur_match.group(1).split("Préleveur :")[0].strip()
            preleveur_part = line.split("Préleveur :")[1].strip() if "Préleveur :" in line else None
            
            # Assign "/" if the part is empty or if we have the chain "DR/dr/Dr/PR/Pr/pr /"
            if not prescripteur_part :
                prescripteur_part = "/"
            if not preleveur_part :
                preleveur_part = "/"
                
            # Assign "/" if the part is empty or contains "DR /", "Dr /", "PR /", "Pr /", "dr /", "pr /".
            if not prescripteur_part or re.match(r'^[DdPp][Rr]?\s*/?$', prescripteur_part):
                prescripteur_part = "/"
            if not preleveur_part or re.match(r'^[DdPp][Rr]?\s*/?$', preleveur_part):
                preleveur_part = "/"
            # Assign "/" if the part contains "not mentioned" or "not indicated".
            if "non mentionné" in prescripteur_part or "non indiqué" in prescripteur_part:
                prescripteur_part = "/"
            if "non mentionné" in preleveur_part or "non indiqué" in preleveur_part:
                preleveur_part = "/"
            
            # Show the parts for "Prescripteur" and "Préleveur"
            print(f"Ligne {line_number + 1}: {line}")
            print("Prescripteur:", prescripteur_part)
            print("Préleveur:", preleveur_part)
            print("")
        
        # Check if there is a match with "Préleveur"
        preleveur_match = preleveur_pattern.match(line)
        if preleveur_match:
            # Divide the line into two parts: before and after the "Préleveur :"
            preleveur_part = preleveur_match.group(1)
            prescripteur_part = line.split("Prescripteur :")[1].strip() if "Prescripteur :" in line else None
            
            # Assign "/" if the part is empty or if we have the chain "DR/dr/Dr/PR/Pr/pr /"
            if not prescripteur_part :
                prescripteur_part = "/"
            if not preleveur_part :
                preleveur_part = "/"
                
            # Show the parts for "Prescripteur" and "Préleveur"
            print(f"Ligne {line_number + 1}: {line}")
            print("Prescripteur:", prescripteur_part)
            print("Préleveur:", preleveur_part)
            print("")

extract_information(all_reports_content)

Ligne 19: Prescripteur : Dr MOLINIER Préleveur : Dr MOLINIER
Prescripteur: Dr MOLINIER
Préleveur: Dr MOLINIER

Ligne 68: Prescripteur : DR ELKAIM Préleveur : DR CHAMBON
Prescripteur: DR ELKAIM
Préleveur: DR CHAMBON

Ligne 103: Prescripteur : DR ELKAIM Préleveur : /
Prescripteur: DR ELKAIM
Préleveur: /

Ligne 138: Prescripteur : PR BERTRAND Préleveur : PR BERTRAND
Prescripteur: PR BERTRAND
Préleveur: PR BERTRAND

Ligne 223: Prescripteur : Dr BONNET Préleveur : Dr BARRE
Prescripteur: Dr BONNET
Préleveur: Dr BARRE

Ligne 258: Prescripteur : DR BACCONNIER Préleveur : /
Prescripteur: DR BACCONNIER
Préleveur: /

Ligne 307: Prescripteur : DR BACCONNIER Préleveur : /
Prescripteur: DR BACCONNIER
Préleveur: /

Ligne 352: Prescripteur : DR BONNET Préleveur : DR TERRASA
Prescripteur: DR BONNET
Préleveur: DR TERRASA

Ligne 401: Prescripteur : DR RAOUX Préleveur : DR LAURANS
Prescripteur: DR RAOUX
Préleveur: DR LAURANS

Ligne 440: Prescripteur : Dr LEBAS Préleveur : Dr LEBAS
Prescripteur: Dr LEBAS
P

Après avoir observées et vérifiées les modifications on va les appliquer dans le texte entier de tout les comptes rendus.

In [17]:
# All pre-treatments are applied to all CRs
def extract_information(text):
    # Regular expressions to identify lines with "Prescripteur" or "Préleveur"
    prescripteur_pattern = re.compile(r'Prescripteur\s*:\s*(.*)')
    preleveur_pattern = re.compile(r'Préleveur\s*:\s*(.*)')
    
    modified_text = ""
    
    for line_number, line in enumerate(text.split('\n')):
         # Check for "Prescripteur" match
        prescripteur_match = prescripteur_pattern.match(line)
        if prescripteur_match:
            # Divide the line into two parts: before and after the "Prescripteur"
            prescripteur_part = prescripteur_match.group(1).split("Préleveur :")[0].strip()
            preleveur_part = line.split("Préleveur :")[1].strip() if "Préleveur :" in line else None
            
            # Assign "/" if the part is empty or contains "DR /", "Dr /", "PR /", "Pr /", "dr /", "pr /".
            if not prescripteur_part or re.match(r'^[DdPp][Rr]?\s*/?$', prescripteur_part):
                prescripteur_part = "/"
            if not preleveur_part or re.match(r'^[DdPp][Rr]?\s*/?$', preleveur_part):
                preleveur_part = "/"
            # Assign "/" if the part contains "not mentioned" or "not indicated".
            if "non mentionné" in prescripteur_part or "non indiqué" in prescripteur_part:
                prescripteur_part = "/"
            if "non mentionné" in preleveur_part or "non indiqué" in preleveur_part:
                preleveur_part = "/"
            
            # Modify line with updated parts
            line = f"Prescripteur : {prescripteur_part} Préleveur : {preleveur_part}"
        
        # Check if there is a match with "Préleveur"
        preleveur_match = preleveur_pattern.match(line)
        if preleveur_match:
            # Divide the line into two parts: before and after the sampler
            preleveur_part = preleveur_match.group(1)
            prescripteur_part = line.split("Prescripteur :")[1].strip() if "Prescripteur :" in line else None
            
            # Assign "/" if the part is empty or contains "DR /", "Dr /", "PR /", "Pr /", "dr /", "pr /".
            if not prescripteur_part or re.match(r'^[DdPp][Rr]?\s*/?$', prescripteur_part):
                prescripteur_part = "/"
            if not preleveur_part or re.match(r'^[DdPp][Rr]?\s*/?$', preleveur_part):
                preleveur_part = "/"
            
            # Modify line with updated parts
            line = f"Prescripteur : {prescripteur_part} Préleveur : {preleveur_part}"
        
        modified_text += line + "\n"
    
    return modified_text

# Extract information and modify text
modified_text = extract_information(all_reports_content)

# Display the modifications
print(modified_text)


------ COMPTE RENDU N°1 ------
SERVICE D’ANATOMIE PATHOLOGIQUE ET DE
NEUROPATHOLOGIE - CHU - TIMONE
TUMOROTHEQUE ET BANQUE DE MUSCLES DE l’AP-HM
Professeur Laurent DANIEL
Pôle 18 : Biologie et Pathologie - N°FINESS : 130783293
264 rue Saint Pierre – 13385 MARSEILLE Cedex 05
Tél. 04 13 42 90 10 – Fax. 04 13 42 90 42 / 04 91 38 44 11
N° NIP :
UF : 0467 RICHARD MARIE-ALETH
-
Pr. RICHARD MARIE-ALETH
*HHH2325367*
CONSULT.PETITE CHIRURGIE DERMATO-TA
APHM TIMONE ADULTES
13385 MARSEILLE
_____________________________________________________________________________________
Prescripteur : Dr MOLINIER Préleveur : Dr MOLINIER
A. EXERESE CUTANEE : JAMBE GAUCHE
Renseignements : Face antéro-interne, lésion pigmentée hétérogène suspecte. Mélanome ? Exérèse sans marge
Exérèse cutanée – Formol – Orientation : non - Dimensions : 40 x 35 x 10 mm – Lésion : ~25 mm
Inclusion : 7 bloc(s) – représentatif(s) : A1 a A7
Description : involution lésionnelle, avec pigment dermique, malgré de multiples plans de cou

Maintenant, on règles un soucis qu'on risque d'avoir à l'extraction : 
- Quand on a un seul protocole dans un compte rendu on remarque que l'on a pas de lettre devant le type du protocole, on va donc effectuer un pré-traitement pour l'ajouter pour "normaliser" les comptes-rendus pour faciliter l'extraction.

In [21]:
def process_cr_content(text):
    # Split the text into individual reports based on the specified delimiter
    reports = re.split(r'(------ COMPTE RENDU N°\d+ ------)', text)
    processed_reports = []

    for report in reports:
        # Find the "Préleveur :" position and process the text after it
        preleveur_index = report.find("Préleveur :")
        if preleveur_index != -1:
            # Extract the text after "Préleveur :" to end of the report
            after_preleveur_text = report[preleveur_index:]
            # Split the text into lines for further processing
            lines = after_preleveur_text.split('\n')
            
            for i, line in enumerate(lines):
                # Check if the line is fully uppercase and not starting with "A-Z. "
                if line.isupper() and not re.match(r"[A-Z]\. ", line) and not re.match(r"CONCLUSION", line) and not re.match(r"HHH", line) and not re.match(r"SERVICE D’ANATOMIE PATHOLOGIQUE ET DE", line) and not re.match(r"NEUROPATHOLOGIE", line) and not re.match(r"UF :", line):
                    # Add "A. " prefix to the line
                    modified_line = "A. " + line
                    # Replace the original line in the report with modified line
                    lines[i] = modified_line
                    # Reconstruct the report with the modified line
                    modified_report = '\n'.join(lines)
                    # Replace the original after_preleveur_text in the report
                    report = report[:preleveur_index] + modified_report
                    break  # Stop after modifying the first matching line
            
        processed_reports.append(report)

    # Join the processed reports back into a single text
    modified_text = ''.join(processed_reports)
    return modified_text

# Read the content of the file already loaded into 'text_content'
# and apply the processing function
modified_text_protocol = process_cr_content(modified_text)

# Output a preview of the modified text
print(modified_text_protocol)  # Adjust the preview length as necessary


------ COMPTE RENDU N°1 ------
SERVICE D’ANATOMIE PATHOLOGIQUE ET DE
NEUROPATHOLOGIE - CHU - TIMONE
TUMOROTHEQUE ET BANQUE DE MUSCLES DE l’AP-HM
Professeur Laurent DANIEL
Pôle 18 : Biologie et Pathologie - N°FINESS : 130783293
264 rue Saint Pierre – 13385 MARSEILLE Cedex 05
Tél. 04 13 42 90 10 – Fax. 04 13 42 90 42 / 04 91 38 44 11
N° NIP :
UF : 0467 RICHARD MARIE-ALETH
-
Pr. RICHARD MARIE-ALETH
*HHH2325367*
CONSULT.PETITE CHIRURGIE DERMATO-TA
APHM TIMONE ADULTES
13385 MARSEILLE
_____________________________________________________________________________________
Prescripteur : Dr MOLINIER Préleveur : Dr MOLINIER
A. EXERESE CUTANEE : JAMBE GAUCHE
Renseignements : Face antéro-interne, lésion pigmentée hétérogène suspecte. Mélanome ? Exérèse sans marge
Exérèse cutanée – Formol – Orientation : non - Dimensions : 40 x 35 x 10 mm – Lésion : ~25 mm
Inclusion : 7 bloc(s) – représentatif(s) : A1 a A7
Description : involution lésionnelle, avec pigment dermique, malgré de multiples plans de cou

Une fois la lettre ajoutée, on fais face à un autre problème :
- Certains compte rendus contiennent "CONCLUSION" et d'autres "CONCLUSION :" on va donc uniformiser cela.
- Il manque la lettre dans la conclusion du protocole unique dans un compte rendu on va donc la rajouter également.

In [22]:
def process_cr_content(text):
    # Defining search motives
    report_split_pattern = r'(------ COMPTE RENDU N°\d+ ------)'
    preleveur_pattern = re.compile(r'Préleveur\s*:', re.IGNORECASE)
    uppercase_line_pattern = re.compile(r'^[A-Z\s]+$')
    conclusion_start_pattern = re.compile(r'\nCONCLUSION *:?')

    # Split content into individual reports
    reports = re.split(report_split_pattern, text, flags=re.DOTALL)
    processed_reports = []

    for i in range(1, len(reports), 2):
        header, content = reports[i], reports[i+1]

        # Modify "CONCLUSION" to uniformly have "CONCLUSION:"
        content = conclusion_start_pattern.sub('\nCONCLUSION :', content)

       # Find and add "A. " after "Préleveur :"
        def add_prefix_to_next_uppercase_line(match):
            text_after_preleveur = match.group(0)
            lines = text_after_preleveur.split('\n')
            for j, line in enumerate(lines[1:], start=1):  # Commence à chercher après "Préleveur :"
                if uppercase_line_pattern.match(line) and not line.startswith("A. "):
                    lines[j] = "A. " + line
                    break
            return '\n'.join(lines)

        content = preleveur_pattern.sub(add_prefix_to_next_uppercase_line, content)

        processed_reports.append(header + content)

    # Join modified reports in a single chain
    modified_text = ''.join(processed_reports)
    return modified_text

# Read the content of the file already loaded into 'text_content'
# and apply the processing function
modified_text_concl = process_cr_content(modified_text_protocol)

# Output a preview of the modified text
print(modified_text_concl)  # Adjust the preview length as necessary

------ COMPTE RENDU N°1 ------
SERVICE D’ANATOMIE PATHOLOGIQUE ET DE
NEUROPATHOLOGIE - CHU - TIMONE
TUMOROTHEQUE ET BANQUE DE MUSCLES DE l’AP-HM
Professeur Laurent DANIEL
Pôle 18 : Biologie et Pathologie - N°FINESS : 130783293
264 rue Saint Pierre – 13385 MARSEILLE Cedex 05
Tél. 04 13 42 90 10 – Fax. 04 13 42 90 42 / 04 91 38 44 11
N° NIP :
UF : 0467 RICHARD MARIE-ALETH
-
Pr. RICHARD MARIE-ALETH
*HHH2325367*
CONSULT.PETITE CHIRURGIE DERMATO-TA
APHM TIMONE ADULTES
13385 MARSEILLE
_____________________________________________________________________________________
Prescripteur : Dr MOLINIER Préleveur : Dr MOLINIER
A. EXERESE CUTANEE : JAMBE GAUCHE
Renseignements : Face antéro-interne, lésion pigmentée hétérogène suspecte. Mélanome ? Exérèse sans marge
Exérèse cutanée – Formol – Orientation : non - Dimensions : 40 x 35 x 10 mm – Lésion : ~25 mm
Inclusion : 7 bloc(s) – représentatif(s) : A1 a A7
Description : involution lésionnelle, avec pigment dermique, malgré de multiples plans de coup

Maintenant on rajoute le "A. " dans les conclusions nécessaire.

In [25]:
# Updated code to account for "CONCLUSION" or "CONCLUSION :" as the start of the conclusion section
def add_prefix_to_conclusion(text):
    # Split the content by reports
    reports = re.split(r'(------ COMPTE RENDU N°\d+ ------)', text)
    processed_reports = []

    for i in range(1, len(reports), 2):
        report_header = reports[i]
        report_content = reports[i + 1]
        
        # Adjusted pattern to identify the end of a conclusion section
        conclusion_end_pattern = r'(\n[A-Z]{2} Le \d{2}/\d{2}/\d{4}|\nLe \d{2}/\d{2}/\d{4})'
        # Updated to match "CONCLUSION" followed by optional ":" and space
        conclusion_pattern = re.compile(r'\nCONCLUSION *:?\n(.*?)(?=' + conclusion_end_pattern + r'|$)', re.DOTALL)
        conclusion_match = conclusion_pattern.search(report_content)
        if conclusion_match:
            conclusion_text = conclusion_match.group(1).strip()
            # If the first line of conclusion does not start with "A. ", add "A. " prefix
            if not conclusion_text.startswith("A. ") :
                modified_conclusion = "A. " + conclusion_text
                # Replace the matched conclusion text with the modified one
                report_content = conclusion_pattern.sub(f'\nCONCLUSION :\n{modified_conclusion}', report_content, 1)
        
        processed_reports.append(report_header + report_content)

    # Join the processed reports back into a single string
    modified_text = ''.join(processed_reports)
    return modified_text

# Apply the function to the content of the CRs
modified_text_concl_2 = add_prefix_to_conclusion(modified_text_concl)

print(modified_text_concl_2) 

------ COMPTE RENDU N°1 ------
SERVICE D’ANATOMIE PATHOLOGIQUE ET DE
NEUROPATHOLOGIE - CHU - TIMONE
TUMOROTHEQUE ET BANQUE DE MUSCLES DE l’AP-HM
Professeur Laurent DANIEL
Pôle 18 : Biologie et Pathologie - N°FINESS : 130783293
264 rue Saint Pierre – 13385 MARSEILLE Cedex 05
Tél. 04 13 42 90 10 – Fax. 04 13 42 90 42 / 04 91 38 44 11
N° NIP :
UF : 0467 RICHARD MARIE-ALETH
-
Pr. RICHARD MARIE-ALETH
*HHH2325367*
CONSULT.PETITE CHIRURGIE DERMATO-TA
APHM TIMONE ADULTES
13385 MARSEILLE
_____________________________________________________________________________________
Prescripteur : Dr MOLINIER Préleveur : Dr MOLINIER
A. EXERESE CUTANEE : JAMBE GAUCHE
Renseignements : Face antéro-interne, lésion pigmentée hétérogène suspecte. Mélanome ? Exérèse sans marge
Exérèse cutanée – Formol – Orientation : non - Dimensions : 40 x 35 x 10 mm – Lésion : ~25 mm
Inclusion : 7 bloc(s) – représentatif(s) : A1 a A7
Description : involution lésionnelle, avec pigment dermique, malgré de multiples plans de coup

Maintenant on a une erreur dans le compte rendu n°HHH2327376. En effet celui ci possède deux protocoles nommés A alors que le premier protocole devrait être A et le second B.
On va alors utiliser une fonction qui considère que la conclusion ne fournit pas d'erreur concernant les nom des protocoles comme c'est le cas pour le compte rendu n°HHH2327376.
Cette fonction va alors recupèrer la conclusion et vérifier si les noms des protocoles correspondent aux protocoles dans le compte rendu et donc si il n'y a pas d'erreur.

In [27]:
def normalize_protocol_names_based_on_conclusion(modified_text):
    # Split the content by reports
    reports = re.split(r'(------ COMPTE RENDU N°\d+ ------)', modified_text)
    processed_reports = []
    protocol_prefixes = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    # Iterate over the reports in pairs
    for i in range(1, len(reports), 2):
        report_header = reports[i]
        report_content = reports[i + 1]
        
        # Determine the number of protocols in the conclusion
        conclusion_end_pattern = r'(\n[A-Z]{2} Le \d{2}/\d{2}/\d{4}|\nLe \d{2}/\d{2}/\d{4})'
        conclusion_pattern = re.compile(r'CONCLUSION :((.*?)(?=' + conclusion_end_pattern + r'|$))', re.DOTALL)
        conclusion_match = conclusion_pattern.search(report_content)
        if conclusion_match:
            conclusion_text = conclusion_match.group(1)
            protocol_count_in_conclusion = len(re.findall(r'\n[A-Z]\. ', conclusion_text))
            
            # Find and normalize protocol names in the report content
            protocol_lines = re.findall(r'^[A-Z]\. .* :', report_content, re.MULTILINE)
            current_protocol = 0
            
            for line in protocol_lines:
                if current_protocol < protocol_count_in_conclusion:
                    new_prefix = protocol_prefixes[current_protocol]
                    new_line = re.sub(r'^[A-Z]\.', f'{new_prefix}.', line)
                    report_content = report_content.replace(line, new_line, 1)
                    current_protocol += 1
        
        processed_reports.append(report_header + report_content)

    # Join the processed reports back into a single string
    normalized_text = ''.join(processed_reports)
    return normalized_text

# Apply the normalization to the entire document
normalized_cr = normalize_protocol_names_based_on_conclusion(modified_text_concl_2)

print(normalized_cr)  

------ COMPTE RENDU N°1 ------
SERVICE D’ANATOMIE PATHOLOGIQUE ET DE
NEUROPATHOLOGIE - CHU - TIMONE
TUMOROTHEQUE ET BANQUE DE MUSCLES DE l’AP-HM
Professeur Laurent DANIEL
Pôle 18 : Biologie et Pathologie - N°FINESS : 130783293
264 rue Saint Pierre – 13385 MARSEILLE Cedex 05
Tél. 04 13 42 90 10 – Fax. 04 13 42 90 42 / 04 91 38 44 11
N° NIP :
UF : 0467 RICHARD MARIE-ALETH
-
Pr. RICHARD MARIE-ALETH
*HHH2325367*
CONSULT.PETITE CHIRURGIE DERMATO-TA
APHM TIMONE ADULTES
13385 MARSEILLE
_____________________________________________________________________________________
Prescripteur : Dr MOLINIER Préleveur : Dr MOLINIER
A. EXERESE CUTANEE : JAMBE GAUCHE
Renseignements : Face antéro-interne, lésion pigmentée hétérogène suspecte. Mélanome ? Exérèse sans marge
Exérèse cutanée – Formol – Orientation : non - Dimensions : 40 x 35 x 10 mm – Lésion : ~25 mm
Inclusion : 7 bloc(s) – représentatif(s) : A1 a A7
Description : involution lésionnelle, avec pigment dermique, malgré de multiples plans de coup

Enfin pour cette fin de pré-traitement pour faciliter l'extraction, on va compter le nombre de page par compte rendu mais aussi le nombre de protocoles.

In [28]:
# Adjust the approach to count the number of protocols based on the conclusion section.
def count_protocols_based_on_conclusion(text):
    cr_results = []
    crs = re.split(r'(------ COMPTE RENDU N°\d+ ------)', text)
    crs = [crs[n:n+2] for n in range(1, len(crs), 2)]  # Pair each header with its content

    # Loop through each CR
    for header, content in crs:
        # Extract CR ID
        cr_id_search = re.search(r'\*([A-Z0-9]+)\*', content)
        cr_id = cr_id_search.group(1) if cr_id_search else "Unknown ID"

        # Count number of pages
        page_count_search = re.search(r'Page \d+/(\d+)', content)
        page_count = int(page_count_search.group(1)) if page_count_search else 1

        # Count number of protocols based on letters (A., B., etc.) in the conclusion
        conclusion_search = re.search(r'CONCLUSION(.*?)((Le \d{2}/\d{2}/\d{4})|Page \d+/\d+)', content, re.DOTALL)
        if conclusion_search:
            conclusion_text = conclusion_search.group(1)
            protocol_count = len(re.findall(r'\b[A-Z]\.', conclusion_text))
        else:
            protocol_count = 0

        cr_results.append({'ID': cr_id, 'N° de page': page_count, 'Nbr de protocole': protocol_count})

    return cr_results

# Re-process the CR content to accurately count protocols based on the conclusion sections
corrected_protocol_counts = count_protocols_based_on_conclusion(normalized_cr)

# Display the corrected counts for a few CRs to verify the correction
corrected_protocol_counts  # Display first 5 CRs for verification

[{'ID': 'HHH2325367', 'N° de page': 1, 'Nbr de protocole': 2},
 {'ID': 'HHH2326092', 'N° de page': 1, 'Nbr de protocole': 1},
 {'ID': 'HHH2326093', 'N° de page': 1, 'Nbr de protocole': 1},
 {'ID': 'HHH2326568', 'N° de page': 2, 'Nbr de protocole': 3},
 {'ID': 'HHH2326775', 'N° de page': 1, 'Nbr de protocole': 1},
 {'ID': 'HHH2327075', 'N° de page': 1, 'Nbr de protocole': 2},
 {'ID': 'HHH2327079', 'N° de page': 1, 'Nbr de protocole': 2},
 {'ID': 'HHH2327227', 'N° de page': 1, 'Nbr de protocole': 3},
 {'ID': 'HHH2327259', 'N° de page': 1, 'Nbr de protocole': 1},
 {'ID': 'HHH2327376', 'N° de page': 1, 'Nbr de protocole': 2},
 {'ID': 'HHH2327378', 'N° de page': 1, 'Nbr de protocole': 3},
 {'ID': 'HHH2327385', 'N° de page': 1, 'Nbr de protocole': 2},
 {'ID': 'HHH2327419', 'N° de page': 2, 'Nbr de protocole': 5},
 {'ID': 'HHH2327422', 'N° de page': 1, 'Nbr de protocole': 2},
 {'ID': 'HHH2327524', 'N° de page': 2, 'Nbr de protocole': 3},
 {'ID': 'HHH2327629', 'N° de page': 1, 'Nbr de protocol

### PARTIE 3 : EXTRACTION DES INFOS

Maintenant, nous allons extraire de chaque compte rendu, chaque protocole et extraire toutes les informations de ces protocoles dans un dictionnaire dédié, on aura alors une paire clé-valeur avec chaque clé correspondant à un champs extrait du protocole.

Evidemment chaque protocole sera différencié par son ID qui correspondra à l'ID du compte rendu finissant par la lettre du protocole.

#####  Ici on a 3 fonctions d'occupant du pied de page :

- extract_footer : qui permet d'extraire le pied de page de chaque rapport en entier.

- extract_footer_columns : permet d'extraire les colonnes suivantes du pied de page de chaque compte rendu :
    - Colonne {DATE}
    - Colonne {SIGNATAIRE}
    - Colonne {PRELIM}
    - Colonne {NB_PAGE}
    - Colonne {SECRETAIRE}
    - Colonne {TOPOMORPHO}
    - Colonne {CHARGE_ITEMS_LISTED_SHORT}


In [29]:
# Creation of 7 columns and fill them with the right informations for each footer of reports.
# The 7 columns are :
# DATE
# SIGNATAIRE
# PRELIM
# NB_PAGE
# SECRETAIRE
# TOPOMORPHO
# CHARGE_ITEMS_LISTED_SHORT
def extract_footer_columns(footer):
    # Define the columns
    columns = {
        "DATE": "/",
        "SIGNATAIRE": "/",
        "PRELIM": [],
        "NB_PAGE": "/",
        "SECRETAIRE": "/",
        "TOPOMORPHO": "/",
        "CHARGE_ITEMS_LISTED_SHORT": "/"
    }

    if footer:
        # Extract DATE
        date_match = re.search(r'Le (\d{2}/\d{2}/\d{4})', footer)
        columns["DATE"] = date_match.group(1) if date_match else "/"

        # Extract SIGNATAIRE
        signataire_match = re.search(r"(Dr|dr|DR|PR|pr|Pr)\.?\s+[A-Za-z]+(?:\s+[A-Za-z]+)?", footer)
        columns["SIGNATAIRE"] = signataire_match.group() if signataire_match else "/"

        # Extract PRELIM
        prelim_matches = re.findall(rf'\bHHH\S+(?=\s)', footer)
        columns["PRELIM"] = prelim_matches

        # Concatenate PRELIM items
        columns["PRELIM"] = " ".join(columns["PRELIM"])

        # Extract NB_PAGE
        nb_page_match = re.search(r'Page\s+(\d+/\d+)', footer)
        columns["NB_PAGE"] = nb_page_match.group(1) if nb_page_match else "/"
        
        # Extract SECRETAIRE
        secretaire_match = re.search(r'([A-Z]{2}) Le \d{2}/\d{2}/\d{4}', footer)
        columns["SECRETAIRE"] = secretaire_match.group(1) if secretaire_match else "/"


        # Extract TOPOMORPHO
        topomorpho_match = re.search(r'\n([A-Z]\d+)\s', footer)
        columns["TOPOMORPHO"] = topomorpho_match.group(1) if topomorpho_match else "/"


        # Extract CHARGE_ITEMS_LISTED_SHORT
        charge_items_match = re.search(r'(1x.+?)(?=\n|$)', footer)
        columns["CHARGE_ITEMS_LISTED_SHORT"] = charge_items_match.group(1) if charge_items_match else "/"

    return columns

# Extract the footer columns from each CR content the footer begins with :
# - "## Le dd/mm/aaaa" OR
# - Le "## dd/mm/aaaa"
# and ends with : "les documents qualités correspondants."
def extract_footer(cr_text):
    # Compiling motifs for the start of the footer
    start_patterns = [
        re.compile(r"[A-Z]{2} Le \d{2}/\d{2}/\d{4}"), # Date with prefix
        re.compile(r"Le \d{2}/\d{2}/\d{4}")  # Date pattern
    ]
    
    # Text at the end of the footer
    end_text = "les documents qualités correspondants."
    
    # Initialize start and end indices to "/"
    start_index = None
    end_index = None
    
    # Search for the beginning of the footer
    for pattern in start_patterns:
        match = pattern.search(cr_text)
        if match:
            start_index = match.start()
            break
    
    # Find the end of the footer
    if start_index is not None:
        end_index = cr_text.find(end_text, start_index)
    
    # Footer extraction if start and end indexes are found
    if start_index is not None and end_index is not None:
        # Adjustment to include end text in extraction
        end_index += len(end_text)
        return cr_text[start_index:end_index]
    # Footer not found 
    else:
        return "/"

#### Ici on a 3 fonctions pour extraire la conclusion :
- extract_conclusion : permet de recupérer la conclusion de chaque rapport pour la stocker dans une variable.
- sort_conclusions_for_protocols : permet à partir de la variable de la conclusion récupérer chaque conclusion de chaque protocole pour les stocker dans une variable .
- process_sorted_conclusions : permet d'extraires 3 colonnes de chaque conclusion de chaque protocoles de chaque CR.

In [30]:
# Creation of 3 columns and fill them with the right informations for each conclusion of each protocol.
# The 3 columns are :
# CONCLU_LOCTYPE_PRELEVEMENT
# CONCLU_DIAGNOSTIC
# CONCLU_LIMITES
def process_sorted_conclusions(sorted_conclusions):
    processed_conclusions = {}
    # Loop through the sorted conclusions
    for letter, conclusion in sorted_conclusions.items():
        # Extraction of the conclusion parts for each protocol
        matches = re.match(rf"^{letter}\.\s*(.*?):\s*(.*?)(?:\s*-\s*limites\s*(.*))?$", conclusion, re.DOTALL)
        # Assign the extracted parts to the corresponding columns
        if matches:
            loc_type_prelevement = matches.group(1).strip()
            diagnostic = matches.group(2).strip()
            limits = matches.group(3).strip() if matches.group(3) else "/"
        # If no match is found, assign a default value
        else:
            loc_type_prelevement = "/"
            diagnostic = "/"
            limits = "/"
        # Add the extracted parts to the processed conclusions dictionary
        processed_conclusions[letter] = {
            'CONCLU_LOCTYPE_PRELEVEMENT': loc_type_prelevement,
            'CONCLU_DIAGNOSTIC': diagnostic,
            'CONCLU_LIMITES': limits
        }
    return processed_conclusions

# The conclusion of a report is sorted to extract each conclusion from each protocol in the report.
# By the regex : "LETTER_OF_THE_PROTOCOL. CONCLUSION_OF_THE_PROTOCOL"
def sort_conclusions_for_protocols(conclusion_text):
    # Dictionary to store the conclusion of each protocol
    conclusions_by_protocol = {}
    # Separation of conclusion into lines
    lines = conclusion_text.split('\n')
    # Regular expression to identify the start of a protocol conclusion
    protocol_start_pattern = re.compile(r"^([A-Z])\.\s")

    current_protocol = ''
    for line in lines:
        match = protocol_start_pattern.match(line)
        if match:
            # New protocol identified
            current_protocol = match.group(1)
            # Initialise ou réinitialise la conclusion pour le protocole actuel
            conclusions_by_protocol[current_protocol] = line + '\n'
        elif current_protocol:
            # Adds the line to the conclusion of the current protocol
            conclusions_by_protocol[current_protocol] += line + '\n'

    # Cleans up conclusions by removing superfluous line breaks
    for protocol, text in conclusions_by_protocol.items():
        conclusions_by_protocol[protocol] = text.strip()

    return conclusions_by_protocol

# The conclusion of a report is extracted, taking into account that it must begin after "CONCLUSION :"
# and end before :
# - Either "The dd/mm/yyyy" 
# - Or "## Le dd/mm/aaaa" (with "##" corresponding to two capital letters)
# - Or "HHH*******A" (with "*" corresponding to a number)
def extract_conclusion(cr_text):
    # Define patterns to identify the start of the conclusion and possible ends
    conclusion_start_pattern = r"CONCLUSION :"
    conclusion_end_patterns = [
        r"HHH\d{7}A;.*",  # Pattern for ID followed by "A"
        r"Le \d{2}/\d{2}/\d{4}",  # Date pattern
        r"[A-Z]{2} Le \d{2}/\d{2}/\d{4}",  # Date with prefix
        r"(Dr|dr|DR|PR|pr|Pr)\. [A-Za-z]+ [A-Za-z]+"  # Name pattern
    ]

    # Find the start of the conclusion
    start_match = re.search(conclusion_start_pattern, cr_text)
    if not start_match:
        return "/"  # Return a placeholder if no conclusion is found

    # Extract the part of the text where the conclusion is expected to be
    conclusion_part = cr_text[start_match.end():]

    # Split the conclusion part into lines for line-by-line pattern checking
    lines = conclusion_part.split('\n')

    # Initialize variables to store the conclusion text and check for the end pattern
    conclusion_text = ""
    for line in lines:
        # Check each pattern on the current line
        if any(re.match(pattern, line) for pattern in conclusion_end_patterns):
            break  # Stop if any pattern matches
        conclusion_text += line + '\n'  # Add the line to the conclusion text

    return conclusion_text.strip()

#### extract_information_from_description : permet d'extraire les colonnes spécifiques à chaque protocoles de la description de chaque protocoles de chaque compte rendu.

In [32]:
# Creation of 23 columns and fill them with the right informations for each description of each protocol.
# The 23 columns are :
# RENSEIGNEMENT
# FIXATEUR
# ORIENTATION
# DIMENSION_X
# DIMENSION_Y
# DIMENSION_Z
# LESION
# INCLUSION
# BLOC_REPRESENTATIF
# BLOC_TECHNIQUE
# DESCRIPTION_HISTO
# BLOC_IHC
# PRONOSTIC
# INTERPRETATION_IHC
# COMMENTAIRE_IHC
# LIMITE_SIMPLE
# HYPOTHESE
# LIMITE_LATERALE
# LIMITE_PROFONDE
# ATTEINTE_LATERALE
# ATTEINTE_PROFONDE
# MARGE_LATERALE
# MARGE_PROFONDE
def extract_information_from_description(description_text, protocol_letter, type_prelevement):
    # Define regex patterns for each required field with modifications to handle dashes and line endings appropriately
    
    liste_fixateurs = ["Formol", "Frais"]
    regex_patterns = {
        'RENSEIGNEMENT_'+ protocol_letter: r"(?:Renseignements|Rens\. Clinique) ?: (.*?)(?=\n|$)",
        'HYPOTHESE_'+ protocol_letter: r"Hypothèse\(s\) ?: (.*?)(?=\n|$)",
        'FIXATEUR_'+ protocol_letter: r"Fixateur(?:s)? ?: (.*?)(?=\n|$)",
        'ORIENTATION_'+ protocol_letter: r"Orientation ?: (.*?)(?= -|\n|$)",
        'DIMENSION_X_'+ protocol_letter: r"Dimensions ?: (\d+(?:,\d+)?) x",
        'DIMENSION_Y_'+ protocol_letter: r"Dimensions ?: \d+(?:,\d+)? x (\d+(?:,\d+)?) x",
        'DIMENSION_Z_'+ protocol_letter: r"Dimensions ?: \d+(?:,\d+)? x \d+(?:,\d+)? x (\d+(?:,\d+)?) (?:cm|mm| -|$)",
        'LESION_'+ protocol_letter: r"Lésion ?: (.*?)(?=\n|$)",
        'INCLUSION_'+ protocol_letter: r"Inclusion(?: totale)? ?: (\d+) bloc(?:s)?",
        'BLOC_REPRESENTATIF_'+ protocol_letter: r"représentatif(?:s)?[^:]*: (.*?)(?= –|\n|$)",
        'BLOC_TECHNIQUE_'+ protocol_letter: r"technique(?:s)? ?: (.*?)(?=\n|$)",
        'DESCRIPTION_HISTO_'+ protocol_letter: r"Description ?: (.*?)(?=Immunohistochimie|Limites|Pronostic|\n[A-Z]\.|$)",
        'BLOC_IHC_'+ protocol_letter: r"Immunohistochimie(?:s)? ?: (.*?)(?=Limites|Interprétation|\n[A-Z]\.|$)",
        'PRONOSTIC_'+ protocol_letter: r"Pronostic :([\s\S]*?)(?=\nLimite|Immunohistochimies|$)",
        'INTERPRETATION_IHC_'+ protocol_letter: r"Interprétation ?: (.*?)(?=\n|$)",
        'COMMENTAIRE_IHC_'+ protocol_letter: r"Commentaires ?: (.*?)(?=Limites|\n[A-Z]\.|$)",
        'LIMITE_SIMPLE_'+ protocol_letter: r"Limites d’exérèse ?: (saines|non-saines|lésionnelles)(?=\s|$)",
        'LIMITE_LATERALE_'+ protocol_letter: r"Limites ?(?: :)? latérales ?(?:\:)? (saines|non-saines)(?=\s-|\n|$)",
        'LIMITE_PROFONDE_'+ protocol_letter: r"- profondes ?(saines|non-saines)(?=\s|$)",
        'ATTEINTE_LATERALE_'+ protocol_letter: r"atteinte latérale ?: (.*?)(?=\n|$)",
        'ATTEINTE_PROFONDE_'+ protocol_letter: r"Atteinte profonde ?: (.*?)(?=\n|$)",
        'MARGE_LATERALE_'+ protocol_letter: r"[Mm]arge latérale ?(?: :)?(.*?)(?=\n|[Mm]arge|$)",
        'MARGE_PROFONDE_'+ protocol_letter: r"[Mm]arge profonde ?(?: :)?(.*?)(?=\n|[Mm]arge|$)",    
    }
    
    # Initialize the dictionary to ensure it's always returned
    extracted_info = {}  
    
    # Extract information from the description section
    for key, pattern in regex_patterns.items():
        match = re.search(pattern, description_text, re.IGNORECASE | re.DOTALL)
        if match:
            extracted_info[key] = match.group(1).strip() if match.group(1) else "/"
        else:
            extracted_info[key] = "/"
            
    # Check for fixative terms in the description
    extracted_info['FIXATEUR_'+ protocol_letter] = "/"
    for term in liste_fixateurs:
        if term in description_text:
            extracted_info['FIXATEUR_'+ protocol_letter] = term
            break
            
    # Extract dimensions with units
    dimensions_matches = re.findall(r"Dimensions ?: (\d+(?:,\d+)?) x (\d+(?:,\d+)?) x (\d+(?:,\d+)?) (cm|mm)", description_text)
    if dimensions_matches:
        extracted_info['DIMENSION_X_'+ protocol_letter] = dimensions_matches[0][0] + " " + dimensions_matches[0][3]
        extracted_info['DIMENSION_Y_'+ protocol_letter] = dimensions_matches[0][1] + " " + dimensions_matches[0][3]
        extracted_info['DIMENSION_Z_'+ protocol_letter] = dimensions_matches[0][2] + " " + dimensions_matches[0][3]
    else:
        extracted_info['DIMENSION_X_'+ protocol_letter] = "/"
        extracted_info['DIMENSION_Y_'+ protocol_letter] = "/"
        extracted_info['DIMENSION_Z_'+ protocol_letter] = "/"

    return extracted_info

#### extract_role : permet d'extraire les rôles de "Prescripteur" ou "Préleveur" de leur ligne pour leur traitement

In [33]:
# Extraction of "Préleveur" and "Prescripteur"
# They should start by :
# - DR/Dr/dr/PR/Pr/pr [name]
# and end by :
# the end of the line for "Préleveur"
# the regex "Préleveur :" for "Prescripteur"
def extract_role(cr_text, role_name):
    # Search for the role in the text
    match = re.search(f"{role_name}\\s*:\\s*([^\\n]*)", cr_text)
    # If found, return the role
    if match:
        role = match.group(1).strip()
        # Remove "Préleveur" from the text of "Prescripteur" because sometimes we have "Prescripteur : DR NOM Préleveur : DR NOM2" in the Prescripteur fields which is not correct.
        if "Préleveur :" in role:
            role = role.split("Préleveur :")[0].strip()
        # Compare the role with the pattern ""
        return role if re.match(r"(DR|Dr|PR|Pr|pr|dr) [^\n]+", role) else "/"
    return "/"

#### extract_footer_header_from_description : permet de supprimer les pieds de page de la fin de la première page et en-tête d'une double page de la description du protocole car elle entrave l'analyse de certains protocoles.

In [34]:
def extract_footer_header_from_description(cr_text):
    # Extract only the body before the conclusion
    parts = cr_text.split("CONCLUSION :")[0]  
    # Split into sections based on protocol headers
    protocols = re.split(r'(\n[A-Z]\. [^:]+ : [^\n]+)', parts)
    # Initialize the variable to store the current protocol letter
    current_protocol_letter = '' 
    # Initialize a dictionary to store descriptive texts for each protocol
    descriptive_texts = {}  
    # Initialize a dictionary to store footer texts for each protocol

    for i, section in enumerate(protocols):
        if i % 2 != 0:
            # This is a protocol header, extract the letter for naming
            current_protocol_letter = re.match(r'\n([A-Z])\.', section).group(1)
        else:
            if current_protocol_letter:
                # Extract the footer and header from the current section if present
                header_footer_match = re.search(r'HHH\d+[A-Z];.*?Examen N° HHH\d+', section, re.DOTALL)
                if header_footer_match:
                    # Remove header and footer text from the section
                    section = section.replace(header_footer_match.group(0), '').strip()
                # Normalize multiple newlines
                section = re.sub(r'\n{2,}', '\n', section).strip()  

                # Start extracting descriptive text directly, without skipping lines
                descriptive_text = section.strip()  # Directly use the section as descriptive text
                descriptive_texts[f'Descriptif_texte_{current_protocol_letter}'] = descriptive_text  # Store descriptive text for the current protocol

    return descriptive_texts  # Return extracted descriptive texts and footer texts

#### extract_information : permet d'extraire les informations structurées à partir de nos rapports cliniques. 

Le texte CR est traité pour extraire des informations générales et spécifiques aux protocoles qui sont ensuite organisées et stockées dans des dictionnaires. 
Enfin, les informations extraites pour chaque CR sont imprimées pour vérifier leur exactitude.

In [39]:
def extract_information(cr_text):
    # Initialize a list to store all extracted information from each CR
    extracted_info = []
    
    ################# FOR THE MAIN BODY (EXCLUDING THE CONCLUSION AND THE FOOTER) #################
    
    # Split the CR text into parts based on the delimiter "CONCLUSION :"
    parts = cr_text.split("CONCLUSION :")
    
    # Extract the main body of the CR (excluding the conclusion part)
    cr_body = parts[0]
    
    ################# ID EXTRACTION WHICH ELPE TO IDENTIFY THE CR #################
    
    # Extract the unique ID for the CR
    id_cr_match = re.search(r"\*([A-Z0-9]+)\*", cr_body)
    id_cr = id_cr_match.group(1) if id_cr_match else "ID not found"

    ################# EXTRACTION OF THE ADDRESS OF THE SERVICE #################
    # Extract the 2 lines for the address
    address_match = re.search(r"^(.*)\n(\d{5} MARSEILLE)", cr_body, re.MULTILINE)
    if address_match:
        address_of_service = address_match.group(1).strip() + ", " + address_match.group(2).strip() 
    else:
        address_of_service = "/"
    
    ################# CLEANING DESCRIPTION #################
    
    # Extract descriptive texts and footnotes from the CR body
    descriptive_texts = extract_footer_header_from_description(cr_body)
    
    ################# CONCLUSION EXTRACTION #################
    
    # Extract conclusion text from the entire CR text
    conclusion_text = extract_conclusion(cr_text)
    
    # Sort conclusions for each protocol
    sorted_conclusions = sort_conclusions_for_protocols(conclusion_text)
    
    # Process sorted conclusions
    processed_conclusions = process_sorted_conclusions(sorted_conclusions)
    
    ################# FOOTER EXTRACTION #################
    
    # Extract footer information from the CR text
    footer = extract_footer(cr_text)
    
    # Extract footer columns from the footer text
    footer_columns = extract_footer_columns(footer)
    
    ################# GENERAL INFOS (ALL PROTOCOLS HAS THE SAME) #################
    
    # Extract general information common to all protocols
    general_info = {
        "UF": re.search(r"UF\s*:\s*([^\n]+)", cr_body).group(1).strip(),
        "CHEF_DE_SERVICE": re.search(r"Pr\.\s*([^\n]+)", cr_body).group(1).strip(),
        "SERVICE": re.search(r"\*\w+\*\n([^\n]+)", cr_body).group(1).strip(),
        "ADRESSE_DU_SERVICE": address_of_service,
        "PRESCRIPTEUR": extract_role(cr_body, "Prescripteur"),
        "PRELEVEUR": extract_role(cr_body, "Préleveur")
    }
    
    ################# SPECIFIC INFO EXTRACTION (EACH PROTOCOLS HAS DIFFERENTS) #################
    
    # Extract information for each protocol in the CR
    protocols = re.findall(r"\n([A-Z])\.\s*([^:]+)\s*:\s*([^\n]+)", cr_body, re.DOTALL)
    for protocol_letter, type_prelevement, loc_prelevement in protocols:
        # Extract description text for the protocol
        description_text = descriptive_texts.get(f'Descriptif_texte_{protocol_letter}', "/")
        
        # Extract columns informations from the description text
        extracted_description_info = extract_information_from_description(description_text, protocol_letter, type_prelevement)
        
        # Extract columns informations from conclusion
        protocol_specific_conclusions = processed_conclusions.get(protocol_letter, {
            'CONCLU_LOCTYPE_PRELEVEMENT': '/',
            'CONCLU_DIAGNOSTIC': '/',
            'CONCLU_LIMITES': '/'
        })
        
        # Construct the soe information dictionary for each protocol
        protocol_info = {
            "ID": f"{id_cr}{protocol_letter}",
            **general_info,
            f"TYPE_DE_PRELEVEMENT_{protocol_letter}": type_prelevement.strip(),
            f"LOC_PRELEVEMENT_{protocol_letter}": loc_prelevement.strip(),
            #f"DESCRIPTIF_TEXTE_{protocol_letter}": descriptive_texts.get(f'Descriptif_texte_{protocol_letter}', "/"),
            **extracted_description_info,
            f"CONCLU_LOCTYPE_PRELEVEMENT_{protocol_letter}": protocol_specific_conclusions['CONCLU_LOCTYPE_PRELEVEMENT'],
            f"CONCLU_DIAGNOSTIC_{protocol_letter}": protocol_specific_conclusions['CONCLU_DIAGNOSTIC'],
            f"CONCLU_LIMITES_{protocol_letter}": protocol_specific_conclusions['CONCLU_LIMITES'],
            #f"PIED_DE_PAGE": extract_footer(cr_text),
            **footer_columns  # Add footer columns to the protocol info
        }
        
        # Append the protocol information to the list of extracted information
        extracted_info.append(protocol_info)
    
    return extracted_info

################# CR PREPARATION FOR THE FUNCTIONS #################

# Split the normalized CR text into individual CRs based on the delimiter "------ COMPTE RENDU N°"
cr_texts = normalized_cr.split("------ COMPTE RENDU N°")[1:]

# Extract information for each CR and store it in a list
all_extracted_info = [extract_information(cr) for cr in cr_texts if cr.strip()]

################# PRINTS THE RESULTS #################

# Print the extracted information for each CR
for cr_index, cr_info in enumerate(all_extracted_info, start=1):
    print(f"Compte Rendu {cr_index}:")
    
    # Print information for each protocol in the CR
    for protocol_info in cr_info:
        print(f"\nProtocole {protocol_info['ID'][-1]} : \n")
        for key, value in protocol_info.items():
            # Ensure descriptive text starts on a new line
            if key.startswith("DESCRIPTIF_TEXTE_"):
                print(f"{key}: \n{value}")
            else:
                print(f"{key}: {value}")
    print("\n")  # Add a blank line for readability

Compte Rendu 1:

Protocole A : 

ID: HHH2325367A
UF: 0467 RICHARD MARIE-ALETH
CHEF_DE_SERVICE: RICHARD MARIE-ALETH
SERVICE: CONSULT.PETITE CHIRURGIE DERMATO-TA
ADRESSE_DU_SERVICE: APHM TIMONE ADULTES, 13385 MARSEILLE
PRESCRIPTEUR: Dr MOLINIER
PRELEVEUR: Dr MOLINIER
TYPE_DE_PRELEVEMENT_A: EXERESE CUTANEE
LOC_PRELEVEMENT_A: JAMBE GAUCHE
RENSEIGNEMENT_A: Face antéro-interne, lésion pigmentée hétérogène suspecte. Mélanome ? Exérèse sans marge
HYPOTHESE_A: /
FIXATEUR_A: Formol
ORIENTATION_A: non
DIMENSION_X_A: 40 mm
DIMENSION_Y_A: 35 mm
DIMENSION_Z_A: 10 mm
LESION_A: ~25 mm
INCLUSION_A: 7
BLOC_REPRESENTATIF_A: A1 a A7
BLOC_TECHNIQUE_A: /
DESCRIPTION_HISTO_A: involution lésionnelle, avec pigment dermique, malgré de multiples plans de coupes observés, seul des
remaniements dermiques superficiels fibro-inflammatoire, néovascularisés avec atrophie épidermique, évocateurs
d’une régression lésionnelle complète sont présents. De manière tout à fait interprétative, l’aspect du stroma
cicatriciel qu

#### create_excel_file : permet de creer un fichier excel avec les informations extraites

In [40]:
# Create excel file with the extracted informations 
def create_excel_file(all_extracted_info, output_file):
    # Create an empty list to store DataFrames for each CR's information
    dfs = []

    # Iterate over each CR's information
    for cr_info in all_extracted_info:
        # Convert the CR's information to a DataFrame and append it to the list
        dfs.append(pd.DataFrame(cr_info))

    # Concatenate all DataFrames along the row axis
    combined_df = pd.concat(dfs, ignore_index=True)

    # Write the combined DataFrame to an Excel file
    combined_df.to_excel(output_file, index=False)
    print(f"Excel file '{output_file}' has been created successfully.")

# Create excel file with the extracted information every 
create_excel_file(all_extracted_info, "extracted_information.xlsx")

Excel file 'extracted_information.xlsx' has been created successfully.


#### Enfin on va effectuer un prétraitement sur le fichier Excel, en remplacant les valeurs nulles par '/'.

In [41]:
import pandas as pd

# Charger le fichier Excel
df = pd.read_excel("extracted_information.xlsx")

# Remplacer les valeurs nulles par "/"
df.fillna("/", inplace=True)

# Enregistrer le DataFrame modifié dans le même fichier Excel
df.to_excel("extracted_information.xlsx", index=False)

print("Les colonnes vides ont été remplacées par '/'.")

Les colonnes vides ont été remplacées par '/'.
